### Utils

In [55]:
import base64
import os
import json
from datetime import datetime
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
import requests
import re
import json
from typing import Dict, Any

def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

def load_latest_sprint_status(base_path):
    """
    Find the latest sprint directory and load the project-sprint-status.md file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    str: Content of the project-sprint-status.md file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return "No sprint directories found."
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-status.md in the latest sprint directory
        status_file = os.path.join(sprint_path, 'project-sprint-status.md')
        
        if os.path.exists(status_file):
            with open(status_file, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return f"project-sprint-status.md not found in {latest_sprint}."
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    
def load_latest_sprint_backlog(base_path):
    """
    Find the latest sprint directory and load the project-sprint-backlog.json file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    dict: Content of the project-sprint-backlog.json file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return {"error": "No sprint directories found."}
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-backlog.json in the latest sprint directory
        backlog_file = os.path.join(sprint_path, 'project-sprint-backlog.json')
        
        if os.path.exists(backlog_file):
            with open(backlog_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        else:
            return {"error": f"project-sprint-backlog.json not found in {latest_sprint}."}
    
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}
    
def export_transcript(state, folder_path):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    transcript = state["transcript"]
    
    filename = state["meeting_type"].replace(" ","_") + str(id) + ".txt"
    
    os.makedirs(os.path.join(folder_path), exist_ok=True)
    
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the string to a text file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(transcript)

def export_state(state, folder_path, filename):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = filename+str(id)+".json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content

def load_from_json(file_path):
    """
    Load data from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file.
    
    Returns:
    dict: A dictionary containing the loaded JSON data.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
def render_mermaid_diagram(diagram_code: str) -> str:
    # Encode the Mermaid code
    encoded_diagram = base64.b64encode(diagram_code.encode('utf-8')).decode('utf-8')
    
    # Make a request to the Mermaid rendering service
    url = f"https://mermaid.ink/img/{encoded_diagram}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Return the URL of the rendered image
        return url
    else:
        # If rendering failed, return the original Mermaid code
        return f"```mermaid\n{diagram_code}\n```"
    
def format_mermaid(input_string):
    # Step 1: Remove redundant "```mermaid" at the start and end
    cleaned_string = input_string.replace('```mermaid', '').replace('```', '')
    
    # Step 2: Replace escaped newlines with actual newlines
    formatted_string = cleaned_string.replace(r'\n', '\n')
    
    # Step 3: Strip any leading/trailing whitespace
    formatted_string = formatted_string.strip()

    return formatted_string	


def export_meeting_history(state, output_file='meeting_history.json'):
    """
    Export the meeting history to a JSON file.
    
    Args:
    state (dict): The state dictionary containing the meeting history.
    output_file (str): The name of the output file. Defaults to 'meeting_history.json'.
    
    Returns:
    None
    """
    meeting_history = state.get("meeting_history", [])
    
    # Ensure meeting_history is a list
    if not isinstance(meeting_history, list):
        meeting_history = [meeting_history]
    
    try:
        with open(output_file, 'w') as file:
            json.dump(meeting_history, file, indent=2)
        print(f"Meeting history exported successfully to {output_file}")
    except IOError:
        print(f"Error: Unable to write to file {output_file}")

    return state  # Return the state to maintain consistency with your workflow


def manage_sprint_folders(state, project_folder):
    """
    Manages sprint folders based on the meeting type.
    Creates a new sprint folder if necessary and adds required files.
    
    :param state: The current state dictionary
    :param project_folder: Path to the project folder
    :return: Updated state with new sprint information
    """
    if "planning" in state.get("meeting_type", "").lower() or "plan" in state.get("meeting_type", "").lower():
        # List all directories in the project folder
        directories = [d for d in os.listdir(project_folder) if os.path.isdir(os.path.join(project_folder, d))]
        
        # Filter and find the highest sprint number
        sprint_numbers = [int(re.findall(r'\d+', d)[0]) for d in directories if d.startswith("sprint") and re.findall(r'\d+', d)]
        
        if sprint_numbers:
            new_sprint_number = max(sprint_numbers) + 1
        else:
            new_sprint_number = 1
        
        # Create new sprint folder
        new_sprint_folder = os.path.join(project_folder, f"sprint{new_sprint_number}")
        os.makedirs(new_sprint_folder, exist_ok=True)
        
        # Create project_sprint_status.md
        status_file_path = os.path.join(new_sprint_folder, "project_sprint_status.md")
        with open(status_file_path, 'w') as status_file:
            status_file.write(f"# Sprint {new_sprint_number} Status\n\nStatus details will be updated here.")
        
        # Create project_sprint_backlog.json
        backlog_file_path = os.path.join(new_sprint_folder, "project_sprint_backlog.json")
        initial_backlog = {}
        with open(backlog_file_path, 'w') as backlog_file:
            json.dump(initial_backlog, backlog_file, indent=2)
        
        # Update state with new sprint information
        state["current_sprint_number"] = new_sprint_number
        state["current_sprint_folder"] = new_sprint_folder
        state["sprint_status_file"] = status_file_path
        state["sprint_backlog_file"] = backlog_file_path
        
        print(f"Created new sprint folder: {new_sprint_folder}")
    else:
        print("Meeting type does not indicate a planning session. No new sprint folder created.")
    
    return state


def load_json(file_path: str) -> Dict[str, Any]:
    """
    Reads a JSON file and returns its contents as a dictionary.

    :param file_path: The path to the JSON file to be read
    :return: A dictionary containing the data from the JSON file
    :raises FileNotFoundError: If the specified file does not exist
    :raises json.JSONDecodeError: If the file is not valid JSON
    """
    try:
        if os.path.getsize(file_path) == 0:
            print(f"Warning: {file_path} is empty.")
            return None
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading {file_path}: {str(e)}")
        raise



def get_latest_sprint_folder(project_folder: str) -> str:
    """
    Scans the project folder for sprint folders and returns the name of the latest sprint folder.

    :param project_folder: Path to the project folder (e.g., 'project1/')
    :return: Name of the latest sprint folder (e.g., 'sprint5'), or None if no sprint folders are found
    """
    # List all items in the project folder
    items = os.listdir(project_folder)

    # Filter for sprint folders and extract their numbers
    sprint_folders = []
    for item in items:
        if os.path.isdir(os.path.join(project_folder, item)):
            match = re.match(r'sprint(\d+)', item, re.IGNORECASE)
            if match:
                sprint_number = int(match.group(1))
                sprint_folders.append((item, sprint_number))

    # Sort sprint folders by number (descending) and return the latest
    if sprint_folders:
        latest_sprint = max(sprint_folders, key=lambda x: x[1])
        print(f"Latest sprint folder found: {latest_sprint[0]}")
        return latest_sprint[0]
    else:
        print("No sprint folders found.")
        return None


def export_markdown(content, file_path):
    """
    Export content to a Markdown file.
    
    Args:
    content (str): The content to be written to the file.
    file_path (str): The path where the file should be saved.
    """
    try:
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Successfully exported Markdown to {file_path}")
    except Exception as e:
        print(f"Error exporting Markdown: {str(e)}")

def export_json(content, file_path):
    """
    Export content to a JSON file, handling nested JSON strings.
    
    Args:
    content (str or dict): The content to be written to the file.
    file_path (str): The path where the file should be saved.
    """
    try:
        # If content is a string, try to parse it as JSON
        if isinstance(content, str):
            content = json.loads(content)
        
        # If content is a list with one item, extract that item
        if isinstance(content, list) and len(content) == 1:
            content = content[0]
        
        # If we still have a string, it might be a nested JSON string
        if isinstance(content, str):
            content = json.loads(content)
        
        # Now we should have a proper Python dict or list
        with open(file_path, 'w') as file:
            json.dump(content, file, indent=2)
        print(f"Successfully exported JSON to {file_path}")
    except json.JSONDecodeError:
        print(f"Error: The content is not valid JSON")
    except IOError:
        print(f"Error: Unable to write to file {file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        
def load_latest_json(folder_path: str) -> Dict[str, Any]:
    """
    Reads the latest JSON file from a given folder and returns its contents as a dictionary.

    :param folder_path: The path to the folder containing JSON files
    :return: A dictionary containing the data from the latest JSON file
    :raises FileNotFoundError: If no JSON files are found in the specified folder
    :raises json.JSONDecodeError: If the latest file is not valid JSON
    """
    try:
        # Get all JSON files in the folder
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        if not json_files:
            raise FileNotFoundError(f"No JSON files found in {folder_path}")
        
        # Find the latest JSON file
        latest_file = max(json_files, key=lambda f: os.path.getmtime(os.path.join(folder_path, f)))
        latest_file_path = os.path.join(folder_path, latest_file)
        
        # Read and parse the latest JSON file
        with open(latest_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded latest JSON from {latest_file_path}")
        return data
    
    except FileNotFoundError:
        print(f"Error: No JSON files found in {folder_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {latest_file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading from {folder_path}: {str(e)}")
        raise

### Prompts

In [40]:
MEETING_NOTE_ANALYZER_PROMPT = """
You are an AI assistant specialized in analyzing meeting notes and extracting key information for project management. Your task is to analyze the provided meeting note and extract relevant updates for the project state, considering the broader context of the company and project.

Company Information:
\n <company data> \n {company_data} \n </company data> \n

Project Overview:
\n <project general> \n {project_general} \n </project general> \n

Project Requirements:
\n <project requirements> \n {project_requirements} \n </project requirements> \n

Employee Profiles:
\n <employee profiles> \n {employee_profiles} \n </employee profiles> \n

Meeting History:
\n <meeting history> \n {meeting_history} \n </meeting history> \n

Meeting Note:
\n <meeting note> \n {meeting_note} \n </meeting note> \n

Please extract the following information:
1. Key decisions made
2. Action items assigned (including who they're assigned to)
3. Progress updates on existing tasks or user stories
4. New tasks or user stories identified
5. Any risks or blockers mentioned
6. Updates to project timeline or milestones
7. Changes in team dynamics or morale
8. How this meeting's outcomes align with or impact overall project goals and requirements

Provide your analysis in a structured JSON format with the following keys:
"key_decisions", "action_items", "progress_updates", "new_tasks", "risks_blockers", "timeline_updates", "team_updates", "project_alignment"

Ensure your response is a valid JSON object. Consider the company context, project requirements, and employee roles when analyzing the meeting outcomes.
"""

PROJECT_STATE_UPDATER_PROMPT = """
You are an AI assistant responsible for updating the overall project state based on recent meeting outcomes. 
Your task is to incorporate the latest meeting analysis into the existing project state, considering the broader context of the company and project.

Company Information:
\n <company data> \n {company_data} \n </company data> \n

Project Overview:
\n <project general> \n {project_general} \n </project general> \n

Project Requirements:
\n <project requirements> \n {project_requirements} \n </project requirements> \n

Employee Profiles:
\n <employee profiles> \n {employee_profiles} \n </employee profiles> \n

Meeting History:
\n <meeting history> \n {meeting_history} \n </meeting history> \n

Current Project State:
Note that this represents the state before the meeting. 
\n <current project state> \n {project_state} \n </current project state> \n

Current Backlog:
Note that this represents the state before the meeting.
\n <current project backlog> \n {project_backlog} \n </current project backlog> \n

Meeting Note:
\n <meeting note> \n {meeting_note} \n </meeting note> \n

Latest Meeting Analysis:
\n <meeting analysis> \n {meeting_analysis} \n </meeting analysis> \n

Please update the project state markdown file with the following structure:
1. Overall project progress (update percentage if applicable)
2. Current phase (update if changed)
3. Key milestones and their status (add new milestones, update existing ones)
4. High-level risks or blockers (add new ones, remove resolved ones)
5. Team morale/health indicator (update based on meeting insights)
6. Alignment with project requirements and goals
7. Key performance indicators (KPIs) relevant to the project

Use chain of thought steps:
1. Identify the mentioned changes in the meeting note
2. Verify if these actions are already in the project. If they are, update their status. If not add them to the project.
3. Update the project state accordingly.

Ensure you maintain the format and provide a brief explanation for each significant change.
Your response should be the complete updated project state.

Consider how the recent meeting outcomes impact the overall project goals, timeline, and requirements. 
Reflect on how well the project is progressing in relation to the company's objectives and the team's capabilities.

"""

BACKLOG_MANAGER_PROMPT = """
You are an AI assistant tasked with managing the project backlog based on recent meeting outcomes. Your job is to update the backlog JSON with new items, modify existing ones, and ensure it reflects the current project state, while considering the broader context of the company and project.

Company Information:
\n <company data> \n {company_data} \n </company data> \n

Project Overview:
\n <project general> \n {project_general} \n </project general> \n

Project Requirements:
\n <project requirements> \n {project_requirements} \n </project requirements> \n

Employee Profiles:
\n <employee profiles> \n {employee_profiles} \n </employee profiles> \n

Meeting History:
\n <meeting history> \n {meeting_history} \n </meeting history> \n

Current Project State:
Note that this represents the state before the meeting. 
\n <current project state> \n {project_state} \n </current project state> \n

Current Backlog:
Note that this represents the state before the meeting.
\n <current project backlog> \n {project_backlog} \n </current project backlog> \n

Meeting Note:
\n <meeting note> \n {meeting_note} \n </meeting note> \n

Latest Meeting Analysis:
\n <meeting analysis> \n {meeting_analysis} \n </meeting analysis> \n

Please update the project backlog JSON with the following actions:
1. Add new user stories or tasks identified in the meeting
2. Update the status of existing items (Not Started, In Progress, Done)
3. Modify priority levels if discussed in the meeting
4. Update estimated effort if re-evaluated
5. Assign tasks to team members if specified in the meeting
6. Ensure backlog items align with overall project goals and requirements
7. Add or update tags/labels to categorize items (e.g., feature, bug, technical debt)
8. Update dependencies between backlog items if discussed

Use chain of thought steps:
1. Identify the mentioned changes in the meeting note
2. Verify if these actions are already in the backlog. If they are, update their status. If not add them to the backlog.
3. Update the project backlog accordingly.

Ensure your response is a valid JSON object representing the complete updated backlog. Each backlog item should have the following structure:

  "id": "US001",
  "title": "User Story Title",
  "description": "Detailed description",
  "status": "Not Started",
  "priority": "High",
  "estimated_effort": "5 story points",
  "assigned_to": "Team Member Name",
  "tags": ["feature", "frontend"],
  "dependencies": ["US002", "US003"]


Consider the skills and roles of team members when assigning tasks. Ensure that the backlog reflects the current project priorities and aligns with the overall project timeline and goals.
"""

### State


In [17]:
from typing import TypedDict
class ProjectUpdateState(TypedDict):
    company_data: str
    project_general: str
    project_requirements: str
    employee_profiles: str

    meeting_note: str

    project_state: str
    meeting_history : Dict[str, Any]
    project_backlog : Dict[str, Any]

    meeting_analysis: str
    updated_project_state: str
    updated_project_backlog: Dict[str, Any]
    
from abc import ABC, abstractmethod

class Graph(ABC):
    @abstractmethod
    def create_graph(self):
        pass
    
    @abstractmethod
    def run_graph(self, project_folder):
        pass

### Node functions

In [46]:
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from  dotenv import load_dotenv
import os

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens= 8192)



def meeting_note_analyzer_node(state: ProjectUpdateState) -> ProjectUpdateState:
        formatted_prompt = MEETING_NOTE_ANALYZER_PROMPT.format(
            company_data=state["company_data"],
            project_general=state["project_general"],
            project_requirements=state["project_requirements"],
            employee_profiles=state["employee_profiles"],
            meeting_history=state["meeting_history"],
            meeting_note=state["meeting_note"]
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content="Analyze the meeting note and extract key information."),
            AIMessage(content="Output:")
        ]
        response = model.invoke(messages)
        state["meeting_analysis"] = response.content
        return state

def project_state_updater_node(state: ProjectUpdateState) -> ProjectUpdateState:
        formatted_prompt = PROJECT_STATE_UPDATER_PROMPT.format(
            company_data=state["company_data"],
            project_general=state["project_general"],
            project_requirements=state["project_requirements"],
            employee_profiles=state["employee_profiles"],

            meeting_history=state["meeting_history"],
            project_state=state["project_state"],
            project_backlog=state["project_backlog"],

            meeting_note=state["meeting_note"],
            meeting_analysis=state["meeting_analysis"]
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content="Update the project state based on the meeting analysis."),
        ]
        response = model.invoke(messages)
        messages.append(AIMessage(content=response.content))
        messages.append(HumanMessage(content="Based on these steps, provide the updated project state. \n IMPORTANT: Respond only with the updated project state, without any other text, Just the markdown text, nothing else."))
        response = model.invoke(messages)
        state["updated_project_state"] = "HealthTrack Pro - Project State\n\n" + str(response.content)
        return state    

def backlog_manager_node(state: ProjectUpdateState) -> ProjectUpdateState:
    try:
        # Remove the {id} placeholder from the BACKLOG_MANAGER_PROMPT
        cleaned_prompt = BACKLOG_MANAGER_PROMPT.replace('{id}', 'id')
        
        formatted_prompt = cleaned_prompt.format(
            company_data=state["company_data"],
            project_general=state["project_general"],
            project_requirements=state["project_requirements"],
            employee_profiles=state["employee_profiles"],

            meeting_history=state["meeting_history"],
            project_state=state["project_state"],
            project_backlog=state["project_backlog"],

            meeting_note=state["meeting_note"],
            meeting_analysis=state["meeting_analysis"],
        )
        messages = [
            SystemMessage(content=formatted_prompt),
            HumanMessage(content="Update the project backlog based on the provided information. ANSWER IN JSON FORMAT, AND ONLY WITH THE JSON"),
            AIMessage(content="Output:")
        ]
        response = model.invoke(messages)
        
        # Attempt to parse the JSON response
        try:
            updated_backlog = json.loads(response.content)
            state["updated_project_backlog"] = json.dumps(updated_backlog, indent=2)
        except json.JSONDecodeError as json_error:
            print(f"Error parsing JSON response: {json_error}")
            print("Raw response content:")
            print(response.content)
            state["updated_project_backlog"] = "Error: Invalid JSON response"
        
        return state
    except Exception as e:
        print(f"Error in backlog_manager_node: {type(e).__name__}: {str(e)}")
        print("Current state keys:")
        for key in state.keys():
            print(f"- {key}")
        
        return state

### Graph

In [22]:
from langgraph.graph import StateGraph, END
class UpdateProjectState(Graph):
    def __init__(self):
        self.workflow = None

    def create_graph(self):
        workflow = StateGraph(ProjectUpdateState)

        workflow.add_node("meeting_note_analyzer", meeting_note_analyzer_node)
        workflow.add_node("project_state_updater", project_state_updater_node)
        workflow.add_node("backlog_manager", backlog_manager_node)

        workflow.add_edge("meeting_note_analyzer", "project_state_updater")
        workflow.add_edge("project_state_updater", "backlog_manager")
        workflow.add_edge("backlog_manager", END)

        workflow.set_entry_point("meeting_note_analyzer")

        self.workflow = workflow.compile()

    def run_graph(self, project_folder: str) -> ProjectUpdateState:
        if self.workflow is None:
            raise ValueError("Graph has not been created. Call create_graph() first.")
        
        previous_state = load_latest_json(folder_path = os.path.join(project_folder,"state-logs"))

        state = initialize_project_update_state(
            project_folder=project_folder,
            previous_state=previous_state
        )

        #state = self.workflow.invoke(state)


        return state
    
    def export_updated_state(self, state: ProjectUpdateState, project_folder: str):
        export_state(state= state, folder_path=os.path.join(project_folder, "state-logs"), filename="update_state")
        export_markdown(state["updated_project_state"], os.path.join(project_folder, "project-state.md"))
        export_json(state["updated_project_backlog"], os.path.join(project_folder, "project-backlog.json"))

def initialize_project_update_state(project_folder: str, previous_state) -> ProjectUpdateState:
    return ProjectUpdateState(
        company_data=load_markdown_to_str(os.path.join(project_folder, "company-general.md")),
        project_general=load_markdown_to_str(os.path.join(project_folder, "project-general.md")),
        project_requirements=load_markdown_to_str(os.path.join(project_folder, "project-requirements.md")),
        employee_profiles=load_markdown_to_str(os.path.join(project_folder, "employee-profiles.md")),

        meeting_note=previous_state["note_final"],
        project_state = load_markdown_to_str(file_path="../../../data_/project1/project-state.md"),
        project_backlog = load_json(file_path="../../../data_/project1/project-backlog.json"),
        meeting_history = load_json(file_path="../../../data_/project1/meeting-history.json"),


        meeting_analysis="",
        updated_project_state="",
        updated_project_backlog="",
    )

### Testing

In [21]:
state = initialize_project_update_state(
    project_folder="../../../data_/project1/",
    previous_state=load_latest_json(folder_path = os.path.join("../../../data_/project1/", "state-logs"))
)
state

Successfully loaded latest JSON from ../../../data_/project1/state-logs/note_taker2.json


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [27]:
meeting_note_analyzer_node(state)


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [32]:
project_state_updater_node(state)


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [33]:
backlog_manager_node(state)


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [70]:
graph = UpdateProjectState()
graph.create_graph()
state = graph.run_graph(project_folder="../../../data_/project1/")
state

In [45]:
graph.export_updated_state(state=state, project_folder="../../../data_/project1/")

Successfully exported Markdown to ../../../data_/project1/project-state.md
Successfully exported Markdown to ../../../data_/project1/project-backlog.md


In [37]:
state["meeting_analysis"] = ""
meeting_note_analyzer_node(state)


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [47]:
state["updated_project_state"] = ""
project_state_updater_node(state)


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [56]:
export_json(state["updated_project_backlog"], os.path.join("../../../data_/project1/", "project-backlog.json"))

Successfully exported JSON to ../../../data_/project1/project-backlog.json
